In [499]:
import json
import requests

import geopandas as gpd
from geojson_rewind import rewind
import numpy as np
import pandas as pd
import plotly.express as px

## 1. Load relevant data

Download the dataset of argetinian amateur radio licenses.

While ENACOM has an API, the dataset is provided by a URL, both in their website as in the API.

In [500]:
# Switch to avoid downnloading XSLX each time I ran
donwload_file = False

# Define the name of the file to be saved locally
filename = 'argentina_ham_callsigns.xls'

if donwload_file:
    # Define the URL with the file to be downlaoded
    url = 'https://datosabiertos.enacom.gob.ar/datasets/197452-listado-de-radioaficionados.download/'

    # Send the petition HTTP Get for obtaining the resource
    data = requests.get(url)

    # Save the file locally
    with open(filename, 'wb') as file:
        file.write(data.content)

# Read CSV as pandas DataFrame
df = pd.read_excel(filename)
df.head()

,Titular de la Licencia,Señal Distintiva,Categoría,Expiración,Provincia,Localidad
0,PROTECCION CIVIL (MIN. SEG. NACION),LU0CD,SUPERIOR,2025-05-21,Ciudad de Buenos Aires,CIUDAD AUTONOMA DE BUENOS AIRES
1,ARTURO JORGE PEYRU,LU1AAA,NOVICIO,2024-05-07,Ciudad de Buenos Aires,CIUDAD AUTONOMA DE BUENOS AIRES
2,CHRISTIAN LUIS DIAZ,LU1AAB,NOVICIO,2024-04-26,Ciudad de Buenos Aires,CIUDAD AUTONOMA DE BUENOS AIRES
3,WENCESLAO BERNARDINO MOREL,LU1AAC,GENERAL,2024-05-16,Ciudad de Buenos Aires,CIUDAD AUTONOMA DE BUENOS AIRES
4,JUAN ANTONIO BILOTA,LU1AAD,NOVICIO,2024-08-14,Ciudad de Buenos Aires,CIUDAD AUTONOMA DE BUENOS AIRES


Loading a previously downloaded GeoJSON with the shapefile of Argentina's provinces (*[Source](https://www.kaggle.com/datasets/pablomgomez21/geojson-file-provincias-argentinas)*)

In [501]:
with open('arg_provinces.geojson', 'rb') as file:
    arg_prov_shape = json.load(file)

Lastly, I'll load a CSV with some basic information of each province in Argentina.
+ **province(index)**: Name of the province.
+ **population**, in number of people (obatined by [provisional 2022 census results](https://www.indec.gob.ar/indec/web/Nivel4-Tema-2-41-165))
+ **area**, in $km^{2}$. [Source](https://en.wikipedia.org/wiki/Provinces_of_Argentina)
+ **earnings**, which refers to the monthly mean individual earning for each province for the month of february 2023. [Source](https://datos.gob.ar/dataset/produccion-salarios-promedio-mediano-por-provincia-sector-actividad/archivo/produccion_8ed16580-cc11-48ba-8ba3-81ab3c95b6da)

In [502]:
prov_data = pd.read_csv('arg_provinces_data.csv', delimiter=';').set_index('province')

# Create a propulation density column
prov_data['pop_density'] = (prov_data['population'] / prov_data['area']).round(2)
prov_data.head()

,population,area,earnings,pop_density
province,,,,
Buenos Aires,17569053,307571,244345,57.12
Catamarca,429556,102602,186419,4.19
Chaco,1142963,99633,186228,11.47
Chubut,603120,224686,368289,2.68
Ciudad Autónoma de Buenos Aires,3120612,203,323670,15372.47


In [503]:
prov_data.index

Index(['Buenos Aires', 'Catamarca', 'Chaco', 'Chubut',
       'Ciudad Autónoma de Buenos Aires', 'Corrientes', 'Córdoba',
       'Entre Ríos', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 'Mendoza',
       'Misiones', 'Neuquén', 'Río Negro', 'Salta', 'San Juan', 'San Luis',
       'Santa Cruz', 'Santa Fe', 'Santiago del Estero',
       'Tierra del Fuego, Antártida e Islas del Atlántico Sur', 'Tucumán'],
      dtype='object', name='province')

## 2. Data Manipulation

In [504]:
# Renaming the columns for ease of use
df = df.rename(columns={
    'Titular de la Licencia': 'license_owner',
    'Señal Distintiva': 'callsign',
    'Categoría': 'category',
    'Expiración': 'expiration_date',
    'Provincia': 'province',
    'Localidad': 'city'
})

# Remove leading or trailing spaces for all object columns
for column in df.columns:
    if df[column].dtype == object:
        df[column] = df[column].str.strip() 

# Apply same format to category and city column
df['category'] = df['category'].str.title()
df['city'] = df['city'].str.title()

# Rename province names to adhere to a standard
df['province'] = df['province'].replace({
    'Provincia de Buenos Aires': 'Buenos Aires',
    'Ciudad de Buenos Aires': 'Ciudad Autónoma de Buenos Aires',
    'Tierra del Fuego A. e I.A.S.': 'Tierra del Fuego, Antártida e Islas del Atlántico Sur'
})

In [505]:
df['province'].unique()

array(['Ciudad Autónoma de Buenos Aires', 'Buenos Aires', 'Santa Fe',
       'Chaco', 'Formosa', 'Córdoba', 'Misiones', 'Entre Ríos', 'Tucumán',
       'Corrientes', 'Mendoza', 'Santiago del Estero', 'Salta',
       'San Juan', 'San Luis', 'Catamarca', 'La Rioja', 'Jujuy',
       'La Pampa', 'Río Negro', 'Chubut', 'Santa Cruz',
       'Tierra del Fuego, Antártida e Islas del Atlántico Sur', 'Neuquén',
       nan], dtype=object)

In [506]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14995 entries, 0 to 14994
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   license_owner    14995 non-null  object        
 1   callsign         14995 non-null  object        
 2   category         14995 non-null  object        
 3   expiration_date  14995 non-null  datetime64[ns]
 4   province         14992 non-null  object        
 5   city             14989 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 703.0+ KB


First, we perform some basic Data Quality operations

In [507]:
# Change dtype of Expiration date colunm
df['expiration_date'] = pd.to_datetime(df['expiration_date'])

# Check number of NaNs
df.isna().sum()

license_owner      0
callsign           0
category           0
expiration_date    0
province           3
city               6
dtype: int64

We have some instances were the province or the city are not defined. While we can't do anything about the city, we can find out the corresponding state by looking at the callsign, in which the suffix (usually by the first letter) responds to the province of the station.

Let's take a look into the missing ones:

In [508]:
df.loc[df['province'].isna(), :]

,license_owner,callsign,category,expiration_date,province,city
5322,JUAN CARLOS CHEHADHI,LU3OAB,Novicio,2023-10-16,NaN,NaN
8205,ESTEBAN MANDUCI,LU6EEM,Novicio,2023-09-14,NaN,NaN
9083,LEONARDO ANTONIO PAGLIARO,LU7DNI,Novicio,2023-07-13,NaN,NaN


We create a function that fills NaN values province based on the callsign structure:

In [509]:
def fix_province_nan(callsign):
    # State is defined by the first letter (and sometimes also second) of the suffix
    first_letter = callsign[3]
    second_letter = callsign[4]

    match first_letter:
        case 'A' | 'B' | 'C':
            return 'Ciudad Autónoma de Buenos Aires'
        case 'D' | 'E':
            return 'Buenos Aires'
        case 'F':
            return 'Santa Fe'
        case 'G':
            if second_letter in [chr(i) for i in range(ord('A'), ord('P'))]:
                return 'Chaco'
            else:
                return 'Formosa'
        case 'H':
            return 'Córdoba'
        case 'I':
            return 'Misiones'
        case 'J':
            return 'Entre Ríos'
        case 'K':
            return 'Tucumán'
        case 'L':
            return 'Corrientes'
        case 'M':
            return 'Mendoza'
        case 'N':
            return 'Santiago del Estero'
        case 'O':
            return 'Salta'
        case 'P':
            return 'San Juan'
        case 'Q':
            return 'San Luis'
        case 'R':
            return 'Catamarca'
        case 'S':
            return 'La Rioja'
        case 'T':
            return 'Jujuy'
        case 'U':
            return 'La Pampa'
        case 'V':
            return 'Río Negro' 
        case 'W':
            return 'Chubut'
        case 'X':
            if second_letter in [chr(i) for i in range(ord('A'), ord('P'))]:
                return 'Santa Cruz'
            else:
                return 'Tierra del Fuego, Antártida e Islas del Atlántico Sur'
        case 'Y':
            return 'Neuquén'
        case 'Z':
            return 'Tierra del Fuego, Antártida e Islas del Atlántico Sur'

df['province'] = df.apply(lambda row: fix_province_nan(row['callsign']) if pd.isna(row['province']) else row['province'], axis=1)

# Check if the function corrected the NaN
print(f'NaNs in "province" column: {df["province"].isna().sum()}')

NaNs in "province" column: 0


## 3. Data Analysis

First of all, I'm going to create a GeoDataFrame. This way, I can link the provinces geography from the GeoJSON with the province data and the amateur radio callsigns database.

Had to rewind GeoJSON because if not, ot rendered properly. More info [here](https://pypi.org/project/geojson-rewind/)

In [510]:
# Rewinded GeoJSON
rewinded = rewind(arg_prov_shape, rfc7946=False)

In [533]:
# Create GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(rewinded)
gdf = gdf.rename(columns={'nombre': 'province'})
gdf = gdf.set_index('province')

# Merge province data previously loaded from CSV
gdf = gdf.merge(prov_data, on='province')
gdf['latitude'] = abs(gdf['geometry'].centroid.y)

# Auxiliary groupby DataFrame
grouped_by_df = df.groupby(['province', 'category']).size().unstack().fillna(0)

# Count number of licenses for each province
gdf['total_licenses'] = df.groupby('province').size()

# Count number of licenses of each category for each province
for category in ['Novicio', 'General', 'Superior', 'Especial']:
    gdf[category.lower()] = grouped_by_df[category].astype(int)

# Generate an additional column with the licenses as percentage of population
gdf['licenses_perc_pop'] = gdf['total_licenses'] / gdf['population']

gdf.head()

,geometry,population,area,earnings,pop_density,latitude,total_licenses,novicio,general,superior,especial,licenses_perc_pop
province,,,,,,,,,,,,
Tucumán,"POLYGON ((-66.20361 -26.62782, -66.11572 -26.2...",1703186,22524,152862,75.62,26.953960,116,49,26,37,4,0.000068
Santiago del Estero,"POLYGON ((-64.42108 -26.28356, -64.22333 -25.6...",1054028,136351,170466,7.73,27.771537,94,62,16,14,2,0.000089
Salta,"POLYGON ((-66.11572 -26.21459, -66.21460 -26.1...",1440672,155488,241973,9.27,24.310458,118,54,33,27,4,0.000082
Jujuy,"POLYGON ((-67.22534 -23.69483, -67.02759 -22.9...",797955,53219,153811,14.99,23.301782,27,17,1,8,1,0.000034
Formosa,"POLYGON ((-62.35840 -24.08659, -62.40234 -22.4...",604278,72066,194757,8.39,24.847571,27,15,8,4,0,0.000045


In [775]:
fig1 = px.bar(gdf['total_licenses'].sort_values(ascending=False), title='Total Amateur Radio Licenses per Province', template='simple_white')
fig1.update_layout(height=700, font_family='Ubuntu', title_font_size=20, xaxis_title=None, yaxis_title=None, showlegend=False)
fig1.update_traces(marker_color='chocolate', marker_line_color='DarkSlateGrey', marker_line_width=1, hovertemplate='Licenses: %{y}<extra></extra>')
fig1.update_layout(hovermode='x unified')
fig1.show()

In [793]:
aux = gdf['total_licenses'].sort_values(ascending=False).cumsum() / df.shape[0]
fig0 = px.line(aux, title='Cumulative Amateru Radio Licenses', template='simple_white', markers=True)
fig0.update_layout(height=700, font_family='Ubuntu', title_font_size=20, xaxis_title=None, yaxis_title=None, showlegend=False)
fig0.update_traces(line_color='goldenrod', line_width=2.5, marker_size=10, marker_color='chocolate', marker_line_color='DarkSlateGrey', marker_line_width=2, hovertemplate='With it %{y:.2%} of Total Licenses<extra></extra>')
fig0.update_layout(hovermode='x unified')
fig0.update_layout(yaxis={'tickformat': ',.0%',
                    'range': [0,1.05]})
fig0.add_hline(y=0.5, opacity=1, line_width=2, line_dash='dash', line_color='navy')
fig0.add_hline(y=0.75, opacity=1, line_width=2, line_dash='dash', line_color='navy')
fig0.add_annotation(text='50% of all licenses',
                  xref='paper', yref='paper',
                  x=1, y=0.50, showarrow=False)
fig0.add_annotation(text='75% of all licenses',
                  xref='paper', yref='paper',
                  x=1, y=0.76, showarrow=False)

fig0.show()

In [784]:
fig2 = px.choropleth_mapbox(gdf, 
                           geojson=gdf.geometry, 
                           locations=gdf.index,
                           color='total_licenses',
                           color_continuous_scale = px.colors.sequential.Oranges,
                           mapbox_style='carto-positron', 
                           center={'lat': -40.7, 'lon': -65.12695},
                           zoom=3,
                           opacity=0.6,
                           labels={'total_licenses': 'Total Licenses'},
                           hover_name=gdf.index,
                           title='Total Amateur Radio Licenses')
fig2.update_layout(height=700, width=900, margin={'b':0}, font_family='Ubuntu', title_font_size=20)
fig2.update_traces(hovertemplate='<b>%{hovertext}</b><br>Total Licenses: %{z}<extra></extra>')
fig2.update_layout(coloraxis=dict(colorbar=dict(orientation='h')))
fig2.show()

In [797]:
fig3 = px.bar((gdf['licenses_perc_pop']).sort_values(ascending=False), title='Amateur Radio Licenses Relative to Population per Province')
fig3.update_layout(height=700, font_family='Ubuntu', title_font_size=20, xaxis_title=None, yaxis_title=None, showlegend=False)
fig3.update_traces(marker_color='chocolate', marker_line_color='DarkSlateGrey', marker_line_width=1, hovertemplate='Licenses: %{y:0.4%}<extra></extra>')
fig3.update_layout(hovermode='x unified')
fig3.update_layout(yaxis={'tickformat': ',.2%'})
                    #'range': [0,1.05]})
fig3.show()

In [798]:
fig4 = px.choropleth_mapbox(gdf, 
                           geojson=gdf.geometry, 
                           locations=gdf.index,
                           color='licenses_perc_pop',
                           color_continuous_scale = px.colors.sequential.Oranges,
                           mapbox_style='carto-positron', 
                           center={'lat': -40.7, 'lon': -65.12695},
                           zoom=3,
                           opacity=0.6,
                           labels={'licenses_perc_pop': 'Licenses as % of population'},
                           title='Amateur Radio Licenses Relative to Population',
                           hover_name=gdf.index)
fig4.update_layout(height=700, width=900, margin={'b':0}, font_family='Ubuntu', title_font_size=20)
fig4.update_traces(hovertemplate='<b>%{hovertext}</b><br>Licenses as % of popoulation: %{z:.4%}<extra></extra>')
fig4.update_layout(coloraxis=dict(colorbar=dict(orientation='h', tickformat=',.2%')))
fig4.show()

In [800]:
fig5 = px.scatter(gdf, x='earnings', y='licenses_perc_pop', trendline='ols', title='Earnings vs. Amateur Radio Licenses Relative to Population', hover_name=gdf.index)
fig5.update_layout(height=700, font_family='Ubuntu', title_font_size=20, xaxis_title='Mean Monthly Earning', yaxis_title='Amateur Radio Licenses Relative to Population', showlegend=False)
fig5.update_traces(marker_color='chocolate', marker_size=10, marker_line_color='DarkSlateGrey', marker_line_width=1)
fig5.update_layout(yaxis={'tickformat': ',.2%'})
#fig5.update_layout(hovermode='x unified')
fig5.show()

In [801]:
gdf_without_outliers = gdf.drop(['Ciudad Autónoma de Buenos Aires', 'Buenos Aires', 'Tierra del Fuego, Antártida e Islas del Atlántico Sur'])
fig6 = px.scatter(gdf_without_outliers, x='area', y='licenses_perc_pop', trendline='ols', title='Province Area vs. Amateur Radio Licenses Relative to Population', hover_name=gdf_without_outliers.index)
fig6.update_layout(height=700, font_family='Ubuntu', title_font_size=20, xaxis_title='Province Area [km^2]', yaxis_title='Amateur Radio Licenses Relative to Population', showlegend=False)
fig6.update_traces(marker_color='chocolate', marker_size=10, marker_line_color='DarkSlateGrey', marker_line_width=1)
fig6.update_layout(yaxis={'tickformat': ',.2%'})
#fig5.update_layout(hovermode='x unified')
fig6.show()

In [802]:
gdf_without_outliers = gdf.drop(['Ciudad Autónoma de Buenos Aires', 'Buenos Aires'])
fig7 = px.scatter(gdf, x='latitude', y='licenses_perc_pop', trendline='ols', title='Latitude vs. Amateur Radio Licenses Relative to Population', hover_name=gdf.index)
fig7.update_layout(height=700, font_family='Ubuntu', title_font_size=20, xaxis_title='Latitude [°S]', yaxis_title='Amateur Radio Licenses Relative to Population', showlegend=False)
fig7.update_traces(marker_color='chocolate', marker_size=10, marker_line_color='DarkSlateGrey', marker_line_width=1)
fig7.update_layout(yaxis={'tickformat': ',.2%'})
fig7.show()

In [803]:
# Get number of licenses for each category
total_per_category = df.groupby('category').size()
stacked_per_category = total_per_category.sort_values(ascending=False).cumsum()

In [804]:
fig7 = px.bar(total_per_category.sort_values(ascending=True), orientation='h', title='National Amateur Radio Licenses by Category')
fig7.update_layout(height=500, width=1200, font_family='Ubuntu', title_font_size=20, xaxis_title=None, yaxis_title=None, showlegend=False)
fig7.update_traces(marker_color='chocolate', marker_line_color='DarkSlateGrey', marker_line_width=1, hovertemplate='Licenses: %{x}<extra></extra>')
fig7.update_layout(hovermode='y unified')
fig7.update_layout(margin={'pad': 10})
fig7.show()

## 4. Conclusions

## 5. Possible future work

1. Geolocation of corrected cities.
2. Download from API names of Argentina, to differentiate institutions from people